# Human Action Recognition with OpenVINO™

This notebook demonstrates live human action recognition with OpenVINO, using the [Action Recognition Models](https://docs.openvino.ai/2020.2/usergroup13.html) from [Open Model Zoo](https://github.com/openvinotoolkit/open_model_zoo), specifically an [Encoder](https://docs.openvino.ai/2020.2/_models_intel_action_recognition_0001_encoder_description_action_recognition_0001_encoder.html) and a [Decoder](https://docs.openvino.ai/2020.2/_models_intel_action_recognition_0001_decoder_description_action_recognition_0001_decoder.html). Both models create a sequence to sequence (`"seq2seq"`) \[1\] system to identify the human activities for [Kinetics-400 dataset](https://deepmind.com/research/open-source/kinetics). The models use the Video Transformer approach with ResNet34 encoder \[2\]. The notebook shows how to create the following pipeline:

<img align='center' src="https://user-images.githubusercontent.com/10940214/148401661-477aebcd-f2d0-4771-b107-4b37f94d0b1e.jpeg" alt="drawing" width="1000"/>

Final part of this notebook shows live inference results from a webcam. Additionally, you can also upload a video file.

**NOTE**: To use a webcam, you must run this Jupyter notebook on a computer with a webcam. If you run on a server, the webcam will not work. However, you can still do inference on a video in the final step.

---

\[1\] seq2seq: Deep learning models that take a sequence of items to the input and output. In this case, input: video frames, output: actions sequence. This `"seq2seq"` is composed of an encoder and a decoder. The encoder captures `"context"` of the inputs to be analyzed by the decoder, and finally gets the human action and confidence.

\[2\] [Video Transformer](https://en.wikipedia.org/wiki/Transformer_(machine_learning_model)) and [ResNet34](https://pytorch.org/vision/main/models/generated/torchvision.models.resnet34.html).



#### Table of contents:

- [Imports](#Imports)
- [The models](#The-models)
    - [Download the models](#Download-the-models)
    - [Load your labels](#Load-your-labels)
    - [Load the models](#Load-the-models)
        - [Model Initialization function](#Model-Initialization-function)
        - [Initialization for Encoder and Decoder](#Initialization-for-Encoder-and-Decoder)
    - [Helper functions](#Helper-functions)
    - [AI Functions](#AI-Functions)
    - [Main Processing Function](#Main-Processing-Function)
    - [Run Action Recognition](#Run-Action-Recognition)


### Installation Instructions

This is a self-contained example that relies solely on its own code.

We recommend  running the notebook in a virtual environment. You only need a Jupyter server to start.
For details, please refer to [Installation Guide](https://github.com/openvinotoolkit/openvino_notebooks/blob/latest/README.md#-installation-guide).

<img referrerpolicy="no-referrer-when-downgrade" src="https://static.scarf.sh/a.png?x-pxid=5b5a4db0-7875-4bfb-bdbd-01698b5b1a77&file=notebooks/action-recognition-webcam/action-recognition-webcam.ipynb" />


In [1]:
%pip install -q "openvino>=2024.0.0" "opencv-python" "tqdm"

Note: you may need to restart the kernel to use updated packages.


## Imports
[back to top ⬆️](#Table-of-contents:)


In [2]:
import collections
import os
import time
from typing import Tuple, List

from pathlib import Path

import cv2
import numpy as np
from IPython import display
import openvino as ov
from openvino.runtime.ie_api import CompiledModel

# Fetch `notebook_utils` module
import requests

r = requests.get(
    url="https://raw.githubusercontent.com/openvinotoolkit/openvino_notebooks/latest/utils/notebook_utils.py",
)
open("notebook_utils.py", "w").write(r.text)
import notebook_utils as utils

## The models
[back to top ⬆️](#Table-of-contents:)

### Download the models
[back to top ⬆️](#Table-of-contents:)

Use the `download_ir_model`, a function from the `notebook_utils` file. It automatically creates a directory structure and downloads the selected model.

In this case you can use `"action-recognition-0001"` as a model name, and the system automatically downloads the two models `"action-recognition-0001-encoder"` and `"action-recognition-0001-decoder"`

> **NOTE**: If you want to download another model, such as `"driver-action-recognition-adas-0002"` (`"driver-action-recognition-adas-0002-encoder"` + `"driver-action-recognition-adas-0002-decoder"`), replace the name of the model in the code below. Using a model outside the list can require different pre- and post-processing. 

In [3]:
# A directory where the model will be downloaded.
base_model_dir = "model"
# The name of the model from Open Model Zoo.
model_name = "action-recognition-0001"
# Selected precision (FP32, FP16, FP16-INT8).
precision = "FP16"
model_path_decoder = f"model/intel/{model_name}/{model_name}-decoder/{precision}/{model_name}-decoder.xml"
model_path_encoder = f"model/intel/{model_name}/{model_name}-encoder/{precision}/{model_name}-encoder.xml"
encoder_url = f"https://storage.openvinotoolkit.org/repositories/open_model_zoo/temp/{model_name}/{model_name}-encoder/{precision}/{model_name}-encoder.xml"
decoder_url = f"https://storage.openvinotoolkit.org/repositories/open_model_zoo/temp/{model_name}/{model_name}-decoder/{precision}/{model_name}-decoder.xml"

if not os.path.exists(model_path_decoder):
    utils.download_ir_model(decoder_url, Path(model_path_decoder).parent)
if not os.path.exists(model_path_encoder):
    utils.download_ir_model(encoder_url, Path(model_path_encoder).parent)

### Load your labels
[back to top ⬆️](#Table-of-contents:)

This tutorial uses [Kinetics-400 dataset](https://deepmind.com/research/open-source/kinetics), and also provides the text file embedded into this notebook. 

> **NOTE**: If you want to run `"driver-action-recognition-adas-0002"` model, replace the `kinetics.txt` file to `driver_actions.txt`.

In [4]:
# Download the text from the openvino_notebooks storage
vocab_file_path = utils.download_file(
    "https://storage.openvinotoolkit.org/repositories/openvino_notebooks/data/data/text/kinetics.txt",
    directory="data",
)

with vocab_file_path.open(mode="r") as f:
    labels = [line.strip() for line in f]

print(labels[0:9], np.shape(labels))

'data\kinetics.txt' already exists.
['abseiling', 'air drumming', 'answering questions', 'applauding', 'applying cream', 'archery', 'arm wrestling', 'arranging flowers', 'assembling computer'] (400,)


### Load the models
[back to top ⬆️](#Table-of-contents:)

Load the two models for this particular architecture, Encoder and Decoder. Downloaded models are located in a fixed structure, indicating a vendor, the name of the model, and a precision.

 1. Initialize OpenVINO Runtime.
 2. Read the network from `*.bin` and `*.xml` files (weights and architecture).
 3. Compile the model for specified device.
 4. Get input and output names of nodes.

Only a few lines of code are required to run the model.

Select device from dropdown list for running inference using OpenVINO

In [5]:
device = utils.device_widget()

device

Dropdown(description='Device:', index=2, options=('CPU', 'GPU', 'AUTO'), value='AUTO')

#### Model Initialization function
[back to top ⬆️](#Table-of-contents:)


In [6]:
# Initialize OpenVINO Runtime.
core = ov.Core()


def model_init(model_path: str, device: str) -> Tuple:
    """
    Read the network and weights from a file, load the
    model on CPU and get input and output names of nodes

    :param:
            model: model architecture path *.xml
            device: inference device
    :retuns:
            compiled_model: Compiled model
            input_key: Input node for model
            output_key: Output node for model
    """

    # Read the network and corresponding weights from a file.
    model = core.read_model(model=model_path)
    # Compile the model for specified device.
    compiled_model = core.compile_model(model=model, device_name=device)
    # Get input and output names of nodes.
    input_keys = compiled_model.input(0)
    output_keys = compiled_model.output(0)
    return input_keys, output_keys, compiled_model

#### Initialization for Encoder and Decoder
[back to top ⬆️](#Table-of-contents:)


In [7]:
# Encoder initialization
input_key_en, output_keys_en, compiled_model_en = model_init(model_path_encoder, device.value)
# Decoder initialization
input_key_de, output_keys_de, compiled_model_de = model_init(model_path_decoder, device.value)

# Get input size - Encoder.
height_en, width_en = list(input_key_en.shape)[2:]
# Get input size - Decoder.
frames2decode = list(input_key_de.shape)[0:][1]

In [8]:
print(list(input_key_de.shape)[0:])

[1, 16, 512]


### Helper functions
[back to top ⬆️](#Table-of-contents:)

Use the following helper functions for preprocessing and postprocessing frames:

1. Preprocess the input image before running the Encoder model. (`center_crop` and `adaptative_resize`)
2. Decode top-3 probabilities into label names. (`decode_output`)
3. Draw the Region of Interest (ROI) over the video. (`rec_frame_display`)
4. Prepare the frame for displaying label names over the video. (`display_text_fnc`)

In [9]:
def center_crop(frame: np.ndarray) -> np.ndarray:
    """
    Center crop squared the original frame to standardize the input image to the encoder model

    :param frame: input frame
    :returns: center-crop-squared frame
    """
    img_h, img_w, _ = frame.shape
    min_dim = min(img_h, img_w)
    start_x = int((img_w - min_dim) / 2.0)
    start_y = int((img_h - min_dim) / 2.0)
    roi = [start_y, (start_y + min_dim), start_x, (start_x + min_dim)]
    return frame[start_y : (start_y + min_dim), start_x : (start_x + min_dim), ...], roi


def adaptive_resize(frame: np.ndarray, size: int) -> np.ndarray:
    """
     The frame going to be resized to have a height of size or a width of size

    :param frame: input frame
    :param size: input size to encoder model
    :returns: resized frame, np.array type
    """
    h, w, _ = frame.shape
    scale = size / min(h, w)
    w_scaled, h_scaled = int(w * scale), int(h * scale)
    if w_scaled == w and h_scaled == h:
        return frame
    return cv2.resize(frame, (w_scaled, h_scaled))


def decode_output(probs: np.ndarray, labels: np.ndarray, top_k: int = 3) -> np.ndarray:
    """
    Decodes top probabilities into corresponding label names

    :param probs: confidence vector for 400 actions
    :param labels: list of actions
    :param top_k: The k most probable positions in the list of labels
    :returns: decoded_labels: The k most probable actions from the labels list
              decoded_top_probs: confidence for the k most probable actions
    """
    top_ind = np.argsort(-1 * probs)[:top_k]
    out_label = np.array(labels)[top_ind.astype(int)]
    decoded_labels = [out_label[0][0], out_label[0][1], out_label[0][2]]
    top_probs = np.array(probs)[0][top_ind.astype(int)]
    decoded_top_probs = [top_probs[0][0], top_probs[0][1], top_probs[0][2]]
    return decoded_labels, decoded_top_probs


def rec_frame_display(frame: np.ndarray, roi) -> np.ndarray:
    """
    Draw a rec frame over actual frame

    :param frame: input frame
    :param roi: Region of interest, image section processed by the Encoder
    :returns: frame with drawed shape

    """

    cv2.line(frame, (roi[2] + 3, roi[0] + 3), (roi[2] + 3, roi[0] + 100), (0, 200, 0), 2)
    cv2.line(frame, (roi[2] + 3, roi[0] + 3), (roi[2] + 100, roi[0] + 3), (0, 200, 0), 2)
    cv2.line(frame, (roi[3] - 3, roi[1] - 3), (roi[3] - 3, roi[1] - 100), (0, 200, 0), 2)
    cv2.line(frame, (roi[3] - 3, roi[1] - 3), (roi[3] - 100, roi[1] - 3), (0, 200, 0), 2)
    cv2.line(frame, (roi[3] - 3, roi[0] + 3), (roi[3] - 3, roi[0] + 100), (0, 200, 0), 2)
    cv2.line(frame, (roi[3] - 3, roi[0] + 3), (roi[3] - 100, roi[0] + 3), (0, 200, 0), 2)
    cv2.line(frame, (roi[2] + 3, roi[1] - 3), (roi[2] + 3, roi[1] - 100), (0, 200, 0), 2)
    cv2.line(frame, (roi[2] + 3, roi[1] - 3), (roi[2] + 100, roi[1] - 3), (0, 200, 0), 2)
    # Write ROI over actual frame
    FONT_STYLE = cv2.FONT_HERSHEY_SIMPLEX
    org = (roi[2] + 3, roi[1] - 3)
    org2 = (roi[2] + 2, roi[1] - 2)
    FONT_SIZE = 0.5
    FONT_COLOR = (0, 200, 0)
    FONT_COLOR2 = (0, 0, 0)
    cv2.putText(frame, "ROI", org2, FONT_STYLE, FONT_SIZE, FONT_COLOR2)
    cv2.putText(frame, "ROI", org, FONT_STYLE, FONT_SIZE, FONT_COLOR)
    return frame


def display_text_fnc(frame: np.ndarray, display_text: str, index: int):
    """
    Include a text on the analyzed frame

    :param frame: input frame
    :param display_text: text to add on the frame
    :param index: index line dor adding text

    """
    # Configuration for displaying images with text.
    FONT_COLOR = (255, 255, 255)
    FONT_COLOR2 = (0, 0, 0)
    FONT_STYLE = cv2.FONT_HERSHEY_DUPLEX
    FONT_SIZE = 0.7
    TEXT_VERTICAL_INTERVAL = 25
    TEXT_LEFT_MARGIN = 15
    # ROI over actual frame
    (processed, roi) = center_crop(frame)
    # Draw a ROI over actual frame.
    frame = rec_frame_display(frame, roi)
    # Put a text over actual frame.
    text_loc = (TEXT_LEFT_MARGIN, TEXT_VERTICAL_INTERVAL * (index + 1))
    text_loc2 = (TEXT_LEFT_MARGIN + 1, TEXT_VERTICAL_INTERVAL * (index + 1) + 1)
    cv2.putText(frame, display_text, text_loc2, FONT_STYLE, FONT_SIZE, FONT_COLOR2)
    cv2.putText(frame, display_text, text_loc, FONT_STYLE, FONT_SIZE, FONT_COLOR)

### AI Functions
[back to top ⬆️](#Table-of-contents:)

<img align='center' src="https://user-images.githubusercontent.com/10940214/148401661-477aebcd-f2d0-4771-b107-4b37f94d0b1e.jpeg" alt="drawing" width="1000"/>

Following the pipeline above, you will use the next functions to:

1. Preprocess a frame before running the Encoder. (`preprocessing`)
2. Encoder Inference per frame. (`encoder`)
3. Decoder inference per set of frames. (`decoder`)
4. Normalize the Decoder output to get confidence values per action recognition label. (`softmax`)

In [10]:
def preprocessing(frame: np.ndarray, size: int) -> np.ndarray:
    """
    Preparing frame before Encoder.
    The image should be scaled to its shortest dimension at "size"
    and cropped, centered, and squared so that both width and
    height have lengths "size". The frame must be transposed from
    Height-Width-Channels (HWC) to Channels-Height-Width (CHW).

    :param frame: input frame
    :param size: input size to encoder model
    :returns: resized and cropped frame
    """
    # Adaptative resize
    preprocessed = adaptive_resize(frame, size)
    # Center_crop
    (preprocessed, roi) = center_crop(preprocessed)
    # Transpose frame HWC -> CHW
    preprocessed = preprocessed.transpose((2, 0, 1))[None,]  # HWC -> CHW
    return preprocessed, roi


def encoder(preprocessed: np.ndarray, compiled_model: CompiledModel) -> List:
    """
    Encoder Inference per frame. This function calls the network previously
    configured for the encoder model (compiled_model), extracts the data
    from the output node, and appends it in an array to be used by the decoder.

    :param: preprocessed: preprocessing frame
    :param: compiled_model: Encoder model network
    :returns: encoder_output: embedding layer that is appended with each arriving frame
    """
    output_key_en = compiled_model.output(0)

    # Get results on action-recognition-0001-encoder model
    infer_result_encoder = compiled_model([preprocessed])[output_key_en]
    return infer_result_encoder


def decoder(encoder_output: List, compiled_model_de: CompiledModel) -> List:
    """
    Decoder inference per set of frames. This function concatenates the embedding layer
    froms the encoder output, transpose the array to match with the decoder input size.
    Calls the network previously configured for the decoder model (compiled_model_de), extracts
    the logits and normalize those to get confidence values along specified axis.
    Decodes top probabilities into corresponding label names

    :param: encoder_output: embedding layer for 16 frames
    :param: compiled_model_de: Decoder model network
    :returns: decoded_labels: The k most probable actions from the labels list
              decoded_top_probs: confidence for the k most probable actions
    """
    # Concatenate sample_duration frames in just one array
    decoder_input = np.concatenate(encoder_output, axis=0)
    # Organize input shape vector to the Decoder (shape: [1x16x512]]
    decoder_input = decoder_input.transpose((2, 0, 1, 3))
    decoder_input = np.squeeze(decoder_input, axis=3)
    output_key_de = compiled_model_de.output(0)
    # Get results on action-recognition-0001-decoder model
    result_de = compiled_model_de([decoder_input])[output_key_de]
    # Normalize logits to get confidence values along specified axis
    probs = softmax(result_de - np.max(result_de))
    # Decodes top probabilities into corresponding label names
    decoded_labels, decoded_top_probs = decode_output(probs, labels, top_k=3)
    return decoded_labels, decoded_top_probs


def softmax(x: np.ndarray) -> np.ndarray:
    """
    Normalizes logits to get confidence values along specified axis
    x: np.array, axis=None
    """
    exp = np.exp(x)
    return exp / np.sum(exp, axis=None)

### Main Processing Function
[back to top ⬆️](#Table-of-contents:)

Running action recognition function will run in different operations, either a webcam or a video file. See the list of procedures below:

1. Create a video player to play with target fps (`utils.VideoPlayer`).
2. Prepare a set of frames to be encoded-decoded.
3. Run AI functions
4. Visualize the results.

In [11]:
def run_action_recognition(
    source: str = "0",
    flip: bool = True,
    use_popup: bool = False,
    compiled_model_en: CompiledModel = compiled_model_en,
    compiled_model_de: CompiledModel = compiled_model_de,
    skip_first_frames: int = 0,
):
    """
    Use the "source" webcam or video file to run the complete pipeline for action-recognition problem
    1. Create a video player to play with target fps
    2. Prepare a set of frames to be encoded-decoded
    3. Preprocess frame before Encoder
    4. Encoder Inference per frame
    5. Decoder inference per set of frames
    6. Visualize the results

    :param: source: webcam "0" or video path
    :param: flip: to be used by VideoPlayer function for flipping capture image
    :param: use_popup: False for showing encoded frames over this notebook, True for creating a popup window.
    :param: skip_first_frames: Number of frames to skip at the beginning of the video.
    :returns: display video over the notebook or in a popup window

    """
    size = height_en  # Endoder input size - From Cell 5_9
    sample_duration = frames2decode  # Decoder input size - From Cell 5_7
    # Select frames per second of your source.
    fps = 30
    player = None
    try:
        # Create a video player.
        player = utils.VideoPlayer(source, flip=flip, fps=fps, skip_first_frames=skip_first_frames)
        # Start capturing.
        player.start()
        if use_popup:
            title = "Press ESC to Exit"
            cv2.namedWindow(title, cv2.WINDOW_GUI_NORMAL | cv2.WINDOW_AUTOSIZE)

        processing_times = collections.deque()
        processing_time = 0
        encoder_output = []
        decoded_labels = [0, 0, 0]
        decoded_top_probs = [0, 0, 0]
        counter = 0
        # Create a text template to show inference results over video.
        text_inference_template = "Infer Time:{Time:.1f}ms,{fps:.1f}FPS"
        text_template = "{label},{conf:.2f}%"

        while True:
            counter = counter + 1

            # Read a frame from the video stream.
            frame = player.next()
            if frame is None:
                print("Source ended")
                break

            scale = 1280 / max(frame.shape)

            # Adaptative resize for visualization.
            if scale < 1:
                frame = cv2.resize(frame, None, fx=scale, fy=scale, interpolation=cv2.INTER_AREA)

            # Select one frame every two for processing through the encoder.
            # After 16 frames are processed, the decoder will find the action,
            # and the label will be printed over the frames.

            if counter % 2 == 0:
                # Preprocess frame before Encoder.
                (preprocessed, _) = preprocessing(frame, size)

                # Measure processing time.
                start_time = time.time()

                # Encoder Inference per frame
                encoder_output.append(encoder(preprocessed, compiled_model_en))

                # Decoder inference per set of frames
                # Wait for sample duration to work with decoder model.
                if len(encoder_output) == sample_duration:
                    decoded_labels, decoded_top_probs = decoder(encoder_output, compiled_model_de)
                    encoder_output = []

                # Inference has finished. Display the results.
                stop_time = time.time()

                # Calculate processing time.
                processing_times.append(stop_time - start_time)

                # Use processing times from last 200 frames.
                if len(processing_times) > 200:
                    processing_times.popleft()

                # Mean processing time [ms]
                processing_time = np.mean(processing_times) * 1000
                fps = 1000 / processing_time

            # Visualize the results.
            for i in range(0, 3):
                display_text = text_template.format(
                    label=decoded_labels[i],
                    conf=decoded_top_probs[i] * 100,
                )
                display_text_fnc(frame, display_text, i)

            display_text = text_inference_template.format(Time=processing_time, fps=fps)
            display_text_fnc(frame, display_text, 3)

            # Use this workaround if you experience flickering.
            if use_popup:
                cv2.imshow(title, frame)
                key = cv2.waitKey(1)
                # escape = 27
                if key == 27:
                    break
            else:
                # Encode numpy array to jpg.
                _, encoded_img = cv2.imencode(".jpg", frame, params=[cv2.IMWRITE_JPEG_QUALITY, 90])
                # Create an IPython image.
                i = display.Image(data=encoded_img)
                # Display the image in this notebook.
                display.clear_output(wait=True)
                display.display(i)

    # ctrl-c
    except KeyboardInterrupt:
        print("Interrupted")
    # Any different error
    except RuntimeError as e:
        print(e)
    finally:
        if player is not None:
            # Stop capturing.
            player.stop()
        if use_popup:
            cv2.destroyAllWindows()

### Run Action Recognition
[back to top ⬆️](#Table-of-contents:)

Find out how the model works in a video file. [Any format supported](https://docs.opencv.org/4.5.1/dd/d43/tutorial_py_video_display.html) by OpenCV will work. You can press the stop button anytime while the video file is running, and it will activate the webcam for the next step.

> **NOTE**: Sometimes, the video can be cut off if there are corrupted frames. In that case, you can convert it. If you experience any problems with your video, use the [HandBrake](https://handbrake.fr/) and select the MPEG format.

if you want to use a web camera as an input source for the demo, please change the value of `USE_WEBCAM` variable to True and specify `cam_id` (the default value is 0, which can be different in multi-camera systems).

In [111]:
import numpy as np
import cv2

y = []
x = []

frames2decode = 1
#Process single entry from dataset
def process_video(video_file, ground_truth):

    
    #TODO: Replace with dataloader
    
    source = video_file

    skip_first_frames = 600
    flip = False
    
    size = height_en  # Endoder input size - From Cell 5_9
    sample_duration = frames2decode  # Decoder input size - From Cell 5_7
    # Select frames per second of your source.
    fps = 30
    player = None
    try:
        cap = cv2.VideoCapture(video_file)
        max_frames = 10
        counter = 0
        encoder_output = []
        while(cap.isOpened()):
            ret, frame = cap.read()
            counter = counter + 1
            if frame is None:
                print("Source ended")
                break
    
            scale = 1280 / max(frame.shape)
    
            # Adaptative resize for visualization.
            if scale < 1:
                frame = cv2.resize(frame, None, fx=scale, fy=scale, interpolation=cv2.INTER_AREA)
                cv2.imshow(frame)
    
            # Select one frame every two for processing through the encoder.
            # After 16 frames are processed, the decoder will find the action,
            # and the label will be printed over the frames.
    
            if counter % 200 == 0:
                # Preprocess frame before Encoder.
                (preprocessed, _) = preprocessing(frame, size)
    
                # Measure processing time.
                start_time = time.time()
    
                # Encoder Inference per frame
                encoder_output=encoder(preprocessed, compiled_model_en)
    
                # Decoder inference per set of frames
                # Wait for sample duration to work with decoder model.
                #if len(encoder_output) == sample_duration:
                x.append(encoder_output)
                y.append(ground_truth)
                encoder_output = []
                max_frames -= 1;
                if max_frames <= 0:
                    break;

                #fps = 1000 / processing_time
    except KeyboardInterrupt:
        print("Interrupted")
        cap.release()
    # Any different error
    except RuntimeError as e:
        print(e)
        cap.release()

    cap.release()

In [157]:
import random
import json


video_file = "https://archive.org/serve/ISSVideoResourceLifeOnStation720p/ISS%20Video%20Resource_LifeOnStation_720p.mp4"

train_dset = json.load(open('audio_features.json', 'r'))

print(len(train_dset))
print(train_dset[0])

def get_features(thing):
    return [thing["danceability"], thing["energy"],thing["tempo"] / float(180),thing["valence"]]

for i in tracklist.values():
    print(i)
    
for i in range(len(train_dset)-1):
    process_video("videos/"+train_dset[i]['video_file'], get_features(train_dset[i+1]))

28
{'track_id': '3CzWF2ZTazx5caTSLckEj5', 'track_name': 'ID - ID', 'danceability': 0.556, 'energy': 0.742, 'tempo': 116.958, 'valence': 0.284, 'video_file': '1.mp4'}
ID - ID
SBTRKT - VOLCA
Ede & Nowa - I Am Wavy [INNERVISIONS]
Alex Medina - La Gustosa
Tri/xon - Let's Go
Yet More - Bounce [INNERVISIONS]
Jimi Jules - ID
Jimi Jules - Don't Break My Heart
MULYA - ID [INNERVISIONS]
Jimi Jules - Trippy Yeah
Hunter/Game & Noah Kaluga - Stars (Mano Le Tough Remix) [JUST THIS]
Shiffer & Paul Brenning - We Care (Dixon Edit)
Deer Jade - Firmament [INNERVISIONS]
Ede & Nowa & Samet Gunal - Paradox [INNERVISIONS]
Âme - Asa [INNERVISIONS]
Jimi Jules - ID
Tri/xon - Workout
Yet More - Tryna Jack My Style [INNERVISIONS]
Âme & Curses - Shadow Of Love [INNERVISIONS]


In [158]:
print(y)

[[0.556, 0.742, 0.6497666666666666, 0.284], [0.556, 0.742, 0.6497666666666666, 0.284], [0.556, 0.742, 0.6497666666666666, 0.284], [0.556, 0.742, 0.6497666666666666, 0.284], [0.556, 0.742, 0.6497666666666666, 0.284], [0.556, 0.742, 0.6497666666666666, 0.284], [0.556, 0.742, 0.6497666666666666, 0.284], [0.556, 0.742, 0.6497666666666666, 0.284], [0.556, 0.742, 0.6497666666666666, 0.284], [0.556, 0.742, 0.6497666666666666, 0.284], [0.727, 0.926, 0.7500555555555555, 0.417], [0.727, 0.926, 0.7500555555555555, 0.417], [0.727, 0.926, 0.7500555555555555, 0.417], [0.727, 0.926, 0.7500555555555555, 0.417], [0.727, 0.926, 0.7500555555555555, 0.417], [0.727, 0.926, 0.7500555555555555, 0.417], [0.727, 0.926, 0.7500555555555555, 0.417], [0.727, 0.926, 0.7500555555555555, 0.417], [0.727, 0.926, 0.7500555555555555, 0.417], [0.727, 0.926, 0.7500555555555555, 0.417], [0.658, 0.662, 0.6834166666666667, 0.254], [0.658, 0.662, 0.6834166666666667, 0.254], [0.658, 0.662, 0.6834166666666667, 0.254], [0.658, 0.

In [164]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset, random_split

# Define a simple regression model
class SimpleRegressionModel(nn.Module):
    def __init__(self):
        super(SimpleRegressionModel, self).__init__()
        self.linear = nn.Linear(512, 32)
        self.linear2 = nn.Linear(32, 4)
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        return self.sigmoid(self.linear2(self.linear(x)))

tx = torch.tensor(x).squeeze()
print(tx.shape)
ty = torch.tensor(y)
print(ty.shape)
print(ty[1][2])

# Create dataset and data loader
dataset = TensorDataset(tx, ty)
train_dataset, val_dataset = random_split(dataset, [.8,.2])
train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=16, shuffle=False)

# Initialize the model, loss function and optimizer
model = SimpleRegressionModel()
criterion = nn.MSELoss()
optimizer = optim.SGD(model.parameters(), lr=0.002)

# Training loop
num_epochs = 5000
for epoch in range(num_epochs):
    
    model.train()
    running_train_loss = 0.0
    for inputs, targets in train_loader:
        optimizer.zero_grad()
        outputs = model(inputs)
        if epoch == 1200:
            print(outputs)
            print(targets)
        loss = criterion(outputs, targets)
        loss.backward()
        optimizer.step()
        running_train_loss += loss.item()
    
    # Compute average training loss for the epoch
    avg_train_loss = running_train_loss / len(train_loader)

    # Validation step
    model.eval()
    running_val_loss = 0.0
    with torch.no_grad():
        for inputs, targets in val_loader:
            outputs = model(inputs)
            loss = criterion(outputs, targets)
            running_val_loss += loss.item()
    
    # Compute average validation loss for the epoch
    avg_val_loss = running_val_loss / len(val_loader)

    if (epoch + 1) % 100 == 0:
        print(f'Epoch [{epoch + 1}/{num_epochs}], Training Loss: {avg_train_loss:.4f}, Validation Loss: {avg_val_loss:.4f}')

# Save the trained model
torch.save(model.state_dict(), 'simple_regression_model.pth')

torch.Size([470, 512])
torch.Size([470, 4])
tensor(0.6498)
Epoch [100/5000], Training Loss: 0.0229, Validation Loss: 0.0237
Epoch [200/5000], Training Loss: 0.0220, Validation Loss: 0.0230
Epoch [300/5000], Training Loss: 0.0215, Validation Loss: 0.0229
Epoch [400/5000], Training Loss: 0.0211, Validation Loss: 0.0228
Epoch [500/5000], Training Loss: 0.0210, Validation Loss: 0.0228
Epoch [600/5000], Training Loss: 0.0210, Validation Loss: 0.0227
Epoch [700/5000], Training Loss: 0.0206, Validation Loss: 0.0227
Epoch [800/5000], Training Loss: 0.0205, Validation Loss: 0.0226
Epoch [900/5000], Training Loss: 0.0205, Validation Loss: 0.0226
Epoch [1000/5000], Training Loss: 0.0203, Validation Loss: 0.0225
Epoch [1100/5000], Training Loss: 0.0202, Validation Loss: 0.0225
Epoch [1200/5000], Training Loss: 0.0201, Validation Loss: 0.0225
tensor([[0.6544, 0.8282, 0.7133, 0.4852],
        [0.6935, 0.8632, 0.7545, 0.4793],
        [0.6594, 0.8266, 0.7212, 0.4873],
        [0.6626, 0.8315, 0.7388,

In [145]:
!pip install onnx onnxruntime

In [166]:
dummy_input = tx[0] # Batch size, Number of features
torch.onnx.export(model, dummy_input, 'simple_regression_model.onnx', input_names=['input'], output_names=['output'], dynamic_axes={'input': {0: 'batch_size'}, 'output': {0: 'batch_size'}})

In [167]:


def new_decoder(encoder_output: List, simple_regression_compiled : CompiledModel) -> List:
    """
    Decoder inference per set of frames. This function concatenates the embedding layer
    froms the encoder output, transpose the array to match with the decoder input size.
    Calls the network previously configured for the decoder model (compiled_model_de), extracts
    the logits and normalize those to get confidence values along specified axis.
    Decodes top probabilities into corresponding label names

    :param: encoder_output: embedding layer for 16 frames
    :param: compiled_model_de: Decoder model network
    :returns: decoded_labels: The k most probable actions from the labels list
              decoded_top_probs: confidence for the k most probable actions
    """
    # Concatenate sample_duration frames in just one array
    decoder_input = np.concatenate(encoder_output, axis=0)
    # Organize input shape vector to the Decoder (shape: [1x16x512]]
    decoder_input = decoder_input.squeeze()
    #output_key_de = simple_regression_compiled.output(0)
    # Get results on action-recognition-0001-decoder model
    result_de = simple_regression_compiled([decoder_input])
    # Normalize logits to get confidence values along specified axis
    #probs = softmax(result_de - np.max(result_de))
    # Decodes top probabilities into corresponding label names
    #decoded_labels, decoded_top_probs = decode_output(probs, labels, top_k=3)
    return result_de

In [175]:

# Decoder initialization
sim_de, sim_out_de, simple_regression_compiled = model_init("simple_regression_model.onnx", device.value)
frames2decode = 1

predictions = [0 for i in range(20)]

def run_ai_dj(use_webcam):
    global predictions
    
    source = "1.mp4"

    skip_first_frames = 0
    flip = True
    size = height_en  # Endoder input size - From Cell 5_9
    sample_duration = frames2decode  # Decoder input size - From Cell 5_7
    # Select frames per second of your source.
    size = height_en  # Endoder input size - From Cell 5_9
    sample_duration = frames2decode  # Decoder input size - From Cell 5_7
    # Select frames per second of your source.
    fps = 30
    player = None
    try:
        # Create a video player.
        player = utils.VideoPlayer(0 if use_webcam else source, flip=flip, fps=fps, skip_first_frames=skip_first_frames)
        # Start capturing.
        player.start()

        processing_times = collections.deque()
        processing_time = 0
        encoder_output = []
        decoded_labels = [0, 0, 0]
        decoded_top_probs = [0, 0, 0]
        counter = 0
        # Create a text template to show inference results over video.
        text_inference_template = "Infer Time:{Time:.1f}ms,{fps:.1f}FPS"
        text_template = "{label},{conf:.2f}%"

        while True:
            counter = counter + 1

            # Read a frame from the video stream.
            frame = player.next()
            if frame is None:
                print("Source ended")
                break

            scale = 1280 / max(frame.shape)

            # Adaptative resize for visualization.
            if scale < 1:
                frame = cv2.resize(frame, None, fx=scale, fy=scale, interpolation=cv2.INTER_AREA)

            # Select one frame every two for processing through the encoder.
            # After 16 frames are processed, the decoder will find the action,
            # and the label will be printed over the frames.

            time.sleep(.1)

            if counter % 60 == 0:
                # Preprocess frame before Encoder.
                (preprocessed, _) = preprocessing(frame, size)

                # Measure processing time.
                start_time = time.time()

                # Encoder Inference per frame
                encoder_output.append(encoder(preprocessed, compiled_model_en))

                # Decoder inference per set of frames
                # Wait for sample duration to work with decoder model.
                if len(encoder_output) == sample_duration:
                    decoded_output= simple_regression_compiled(torch.tensor(encoder_output).squeeze())
                    encoder_output = []
                    print(decoded_output)
                    predictions.append(decoded_output[0])
                    predictions.remove(predictions[0])

                # Inference has finished. Display the results.
                stop_time = time.time()

                # Calculate processing time.
                processing_times.append(stop_time - start_time)

                # Use processing times from last 200 frames.
                if len(processing_times) > 200:
                    processing_times.popleft()

                # Mean processing time [ms]
                processing_time = np.mean(processing_times) * 1000
                fps = 1000 / processing_time
                


    # ctrl-c
    except KeyboardInterrupt:
        print("Interrupted")
        player.close()
    # Any different error
    except RuntimeError as e:
        print(e)

In [ ]:
run_ai_dj(True)

{<ConstOutput: names[output] shape[4] type: f32>: array([0.6669922 , 0.80322266, 0.7089844 , 0.44921875], dtype=float32)}
{<ConstOutput: names[output] shape[4] type: f32>: array([0.66259766, 0.8051758 , 0.69921875, 0.48632812], dtype=float32)}
{<ConstOutput: names[output] shape[4] type: f32>: array([0.6616211 , 0.80126953, 0.69921875, 0.49316406], dtype=float32)}
{<ConstOutput: names[output] shape[4] type: f32>: array([0.65966797, 0.80322266, 0.70214844, 0.48583984], dtype=float32)}
{<ConstOutput: names[output] shape[4] type: f32>: array([0.62060547, 0.7705078 , 0.68652344, 0.4951172 ], dtype=float32)}
{<ConstOutput: names[output] shape[4] type: f32>: array([0.6220703 , 0.77685547, 0.69140625, 0.48754883], dtype=float32)}
{<ConstOutput: names[output] shape[4] type: f32>: array([0.6225586 , 0.77001953, 0.6870117 , 0.49414062], dtype=float32)}
{<ConstOutput: names[output] shape[4] type: f32>: array([0.6303711 , 0.75634766, 0.6870117 , 0.48291016], dtype=float32)}
{<ConstOutput: names[out